Our project objective was to train an XGBoost model on energy usage features, GDP, population, and policy indicators to allow a user to simulate how altering these features would influence CO2 emissions for the most recent year for which data was collected (2023). The datasets were obtained from www.census.gov and the US Energy Information Administration. The purpose of this notebook is to load all datasets, then format the dataframes so that they can be merged (For every year (2000-2023), there must be one row per state).

In [66]:
import pandas as pd

'''
Importing all datasets
- electricity sales
- general energy use
- renewables use
- sector use
- state GDP
- total CO2
- CO2 intensity
- RPS policy
- population
'''


electricity_sales = pd.read_excel("electricity_sales.xlsx", sheet_name="Total", header=2)
coal_use = pd.read_excel("general_energy_use.xlsx", sheet_name="Coal", header=2)
ng_use = pd.read_excel("general_energy_use.xlsx", sheet_name="Natural gas", header=2)
petrol_use = pd.read_excel("general_energy_use.xlsx", sheet_name="Petroleum", header=2)
nuclear_use = pd.read_excel("general_energy_use.xlsx", sheet_name="Nuclear", header=2)
renewables_use = pd.read_excel("general_energy_use.xlsx", sheet_name="Total renewable energy", header=2)

biomass_use = pd.read_excel("renewables_use.xlsx", sheet_name="Biomass", header=2)
geothermal_use = pd.read_excel("renewables_use.xlsx", sheet_name="Geothermal", header=2)
hydro_use = pd.read_excel("renewables_use.xlsx", sheet_name="Hydropower", header=2)
solar_use = pd.read_excel("renewables_use.xlsx", sheet_name="Solar energy", header=2)
wind_use = pd.read_excel("renewables_use.xlsx", sheet_name="Wind energy", header=2)

residential_use = pd.read_excel("sector_use.xlsx", sheet_name="Residential sector", header=2)
commercial_use = pd.read_excel("sector_use.xlsx", sheet_name="Commercial sector", header=2)
industrial_use = pd.read_excel("sector_use.xlsx", sheet_name="Industrial sector", header=2)
transportation_use = pd.read_excel("sector_use.xlsx", sheet_name="Transportation sector", header=2)
total_consumption = pd.read_excel("sector_use.xlsx", sheet_name="Total consumption", header=2)

real_gdp = pd.read_excel("real_gdp.xlsx", sheet_name="Real GDP", header=2)
total_co2 = pd.read_excel("CO2_total.xlsx", sheet_name="Total CO2", header=2)
carbon_intensity_energy = pd.read_excel("CO2_total.xlsx", sheet_name="CO2 per billion Btu", header=2)
rps = pd.read_excel("Policy.xlsx", sheet_name = "RPS_Numeric")

pop1 = pd.read_excel("pop_2000-2009.xlsx", header=3)
pop2 = pd.read_excel("pop_2010_2019.xlsx", header=3)
pop3 = pd.read_excel("pop_2020_2023.xlsx", header=3)

In [67]:
'Electricity total consumption (electricity sales to ultimate customers), million kilowatthours'
electricity_sales = electricity_sales.drop(columns=[col for col in electricity_sales.columns if str(col).isdigit() and int(col) < 2000])
electricity_sales.head()

,State,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,AK,5310,5454,5465,5564,5788,5913,6182,6327,6326,...,6165,6159,6123,6186,5972,5819,5918,5969,6002,6025
1,AL,83524,79358,83067,83844,86871,89202,90678,91828,89707,...,90494,88846,88225,86242,90280,88095,83396,85585,87028,84880
2,AR,41611,41732,42450,43108,43672,46165,46636,47055,46135,...,47080,46465,46188,46086,49603,48093,45851,48663,48998,48649
3,AZ,61130,62274,62601,64080,66933,69391,73253,77193,76268,...,76298,77349,78238,77646,78346,77929,81960,81220,84197,85919
4,CA,244057,247759,235213,243221,252026,254250,262959,264235,268155,...,262585,261170,256847,257268,255224,250379,250175,247250,251869,239480


In [68]:
'Coal total consumption, billion Btu'
coal_use = coal_use.drop(columns=[col for col in coal_use.columns if str(col).isdigit() and int(col) < 2000])
coal_use.head()

,State,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,AK,16455,15911,16429,12552,14075,13994,14981,13662,14705,...,18225,19511,16613,16376,17325,17620,18527,18694,18615,18414
1,AL,904181,842337,845999,873735,853917,890098,886678,888436,842826,...,575912,494311,410168,378890,377152,317180,256687,309791,297654,224926
2,AR,267579,273982,255242,253653,270210,247169,256907,275037,278849,...,339214,226889,246437,267591,304145,239798,161980,216123,211724,180262
3,AZ,432810,424023,406479,406537,425413,428445,431985,438524,458727,...,447849,385822,323878,334512,331492,257727,156821,160299,154007,137885
4,CA,70012,67839,70010,69469,68909,67422,67017,66495,63050,...,39486,30967,32077,33663,33256,30886,28034,28244,30049,28746


In [69]:
'Natural gas total consumption (excluding supplemental gaseous fuels), billion Btu'
ng_use = ng_use.drop(columns=[col for col in ng_use.columns if str(col).isdigit() and int(col) < 2000])
ng_use.head()

,State,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,AK,437972,413049,420808,415891,407944,434704,375719,372186,343938,...,329274,333936,330883,343899,346255,343384,374561,395590,437916,448087
1,AL,368472,344004,389965,360483,391927,363398,402044,430649,414260,...,650570,701582,715033,681208,771194,748731,714871,739891,787300,775747
2,AR,256114,231590,247899,254571,217921,216600,240885,229610,238424,...,273009,296826,315617,317530,366939,371388,335436,360545,398099,399566
3,AZ,208137,244424,255247,275650,356264,329302,365230,401992,409966,...,315916,365313,373916,334629,400531,484249,513540,484962,468038,537151
4,CA,2456438,2513856,2318656,2317062,2462247,2304463,2375929,2467545,2472612,...,2409574,2384061,2248939,2190994,2209801,2218732,2153254,2172757,2131372,2154533


In [70]:
'All petroleum products total consumption, excluding biofuels, billion Btu'
petrol_use = petrol_use.drop(columns=[col for col in petrol_use.columns if str(col).isdigit() and int(col) < 2000])
petrol_use.head()

,State,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,AK,275255,289334,277400,285417,334121,333091,342272,323089,278725,...,226704,236497,220481,217142,216500,221845,223361,261094,263335,270391
1,AL,559735,534772,562366,561731,609981,606840,611265,601179,566323,...,491062,512940,540018,531216,519594,530699,527241,583042,578431,565754
2,AR,357586,353382,356282,357343,361081,358312,358679,359757,354937,...,325740,315144,323431,321820,327170,331550,316437,328271,327813,327465
3,AZ,502693,514052,526176,537072,558357,573763,582623,571417,542166,...,547040,561290,581762,588253,594895,611851,549587,606862,594859,599712
4,CA,3571465,3585715,3704382,3520087,3670782,3741092,3786528,3795097,3553537,...,3063217,3168455,3283869,3364148,3388089,3371632,2658831,2959389,3017944,2996168


In [71]:
'Nuclear energy consumed for electricity generation, total, billion Btu'
nuclear_use = nuclear_use.drop(columns=[col for col in nuclear_use.columns if str(col).isdigit() and int(col) < 2000])
nuclear_use.head()

,State,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,AK,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AL,327143,317019,332650,330137,329898,330761,332992,360036,407551,...,431368,438728,417334,446097,412584,455865,454930,480115,442093,476392
2,AR,121516,154356,152024,153093,161111,142864,158952,162434,148085,...,151428,144716,140370,132736,132996,141750,157349,141372,149654,156492
3,AZ,316839,299966,322260,297872,293158,269327,250568,280920,305726,...,338044,340153,338636,338246,325122,333312,329597,329868,333738,329474
4,CA,366845,346911,358707,370958,315634,377313,333488,375427,339506,...,177656,193529,197754,187227,190422,168799,169838,171842,183814,185192


In [72]:
'Renewable energy total consumption, billion Btu'
renewables_use = renewables_use.drop(columns=[col for col in renewables_use.columns if str(col).isdigit() and int(col) < 2000])
renewables_use.head()

,State,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,AK,5574,8119,8526,8974,8964,6220,5330,5674,5363,...,11523,13526,14371,13136,13694,12952,9757,9597,10410,10087
1,AL,223807,195008,193946,199714,223107,213168,219762,202747,198348,...,236143,225388,217520,229887,233969,232118,235538,239817,232035,222189
2,AR,92540,76385,85421,90111,88907,92426,90520,100786,96258,...,115127,107425,105871,103841,105007,106732,91389,89714,90825,87277
3,AZ,45505,39755,37807,36684,36362,49885,51225,54591,64586,...,72405,79520,83630,88295,92790,92495,90813,99266,101215,108445
4,CA,370066,325466,355371,406651,423563,439340,464148,403528,397934,...,560635,564184,645141,724945,690792,788792,738585,810020,880995,1065179


In [73]:
'Biomass total consumption, billion Btu'
biomass_use = biomass_use.drop(columns=[col for col in biomass_use.columns if str(col).isdigit() and int(col) < 2000])
biomass_use.head()

,State,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,AK,2069,3438,3529,3485,3765,1133,1065,1177,1233,...,5565,7438,7939,6849,7288,6723,3088,3153,3846,3404
1,AL,203782,166334,163687,156372,186728,178423,194872,188474,177230,...,203620,191516,193342,197510,194478,191627,188468,198543,193932,189040
2,AR,83466,66836,72970,80405,75931,81469,84735,89208,79733,...,105231,94352,92703,92785,93098,90911,73832,72939,74816,71311
3,AZ,13376,10381,9306,9563,9683,25264,25217,29144,36482,...,31894,35614,35376,36026,38188,38952,33132,35287,36364,36572
4,CA,164105,164137,171500,205815,228444,226207,220118,225986,230787,...,349076,343853,348885,357504,356648,404977,397310,462829,497652,612607


In [74]:
'Geothermal total consumption, billion Btu'
geothermal_use = geothermal_use.drop(columns=[col for col in geothermal_use.columns if str(col).isdigit() and int(col) < 2000])
geothermal_use.head()

,State,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,AK,86,86,86,89,88,90,87,87,130,...,186,186,186,186,186,186,186,186,186,186
1,AL,62,65,67,62,68,78,85,98,117,...,141,141,141,141,141,141,141,141,141,141
2,AR,192,212,228,282,306,349,401,482,572,...,808,808,808,808,808,808,808,808,808,808
3,AZ,286,287,294,235,255,283,287,302,366,...,345,345,345,345,345,345,345,345,345,345
4,CA,44038,43771,46830,46164,46701,46595,45889,46516,46151,...,43430,42684,41231,41580,41980,39378,40921,40106,40288,40578


In [75]:
'Hydropower total consumption, billion Btu'
hydro_use = hydro_use.drop(columns=[col for col in hydro_use.columns if str(col).isdigit() and int(col) < 2000])
hydro_use.head()

,State,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,AK,3418,4591,4911,5400,5111,4995,4175,4406,3998,...,5250,5353,5661,5608,5678,5539,6019,5763,5846,6051
1,AL,19850,28512,30110,43213,36257,34613,24743,14112,20937,...,32301,33648,23832,31518,38020,38915,45547,39309,34762,28762
2,AR,8088,8695,11723,9058,12431,10518,5291,11044,15901,...,9007,12178,12181,10043,10266,14108,15458,13746,11835,11017
3,AZ,28505,26012,25342,24140,23792,21871,23177,22511,24859,...,20876,22300,24456,23310,23823,21170,21917,20379,18075,20258
4,CA,130795,87149,106252,124097,116489,135224,163938,93242,82324,...,56405,47115,98751,144543,89840,130866,72940,50080,60202,110485


In [76]:
'Solar energy total consumption, billion Btu'
solar_use = solar_use.drop(columns=[col for col in solar_use.columns if str(col).isdigit() and int(col) < 2000])
solar_use.head()

,State,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,AK,0,0,0,0,0,0,0,0,2,...,3,3,7,10,13,17,26,45,57,67
1,AL,113,97,82,68,54,53,62,63,64,...,81,82,205,717,1330,1435,1383,1823,3200,4246
2,AR,795,642,500,367,239,90,94,53,53,...,81,87,179,206,836,906,1290,2221,3365,4141
3,AZ,3339,3075,2865,2747,2632,2467,2544,2634,2879,...,17693,19718,21605,26670,28624,30137,33221,37795,41092,45356
4,CA,19125,18469,17815,17284,17238,16771,17543,18727,20299,...,67394,88805,110181,137566,154475,166707,181068,205221,232908,253676


In [77]:
'Wind energy total consumption, billion Btu'
wind_use = wind_use.drop(columns=[col for col in wind_use.columns if str(col).isdigit() and int(col) < 2000])
wind_use.head()

,State,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,AK,0,3,0,0,0,2,3,3,0,...,518,545,578,483,529,487,439,451,475,380
1,AL,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AR,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,AZ,0,0,0,0,0,0,0,0,0,...,1597,1542,1848,1945,1809,1892,2198,5460,5338,5914
4,CA,12003,11941,12975,13291,14692,14543,16660,19056,18373,...,44330,41727,46093,43752,47850,46864,46345,51784,49945,47833


In [78]:
'Total energy consumption in the residential sector, billion Btu'
residential_use = residential_use.drop(columns=[col for col in residential_use.columns if str(col).isdigit() and int(col) < 2000])
residential_use.head()

,State,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,AK,48404,50813,48588,49016,52350,50160,56824,50947,51126,...,44417,49764,47703,50014,46335,45576,46845,48550,46863,48754
1,AL,370236,346578,365195,348234,356075,367927,374411,387829,378548,...,365119,336211,331716,310724,342255,332232,315576,325823,336143,319973
2,AR,218879,214597,216149,211117,206123,224943,220872,225012,226002,...,241530,221167,207237,196907,229627,219617,207867,218637,221257,209353
3,AZ,314800,323477,318457,330484,342228,353479,368093,380183,371463,...,356824,362293,364794,369932,375585,382032,404889,384892,405064,408978
4,CA,1300566,1322525,1304814,1321295,1363294,1311193,1331067,1360526,1400434,...,1212364,1201311,1170025,1170581,1208295,1209812,1259697,1224642,1207171,1166624


In [79]:
'Total energy consumption in the commercial sector, billion Btu'
commercial_use = commercial_use.drop(columns=[col for col in commercial_use.columns if str(col).isdigit() and int(col) < 2000])
commercial_use.head()

,State,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,AK,66776,62031,55830,53845,59006,58006,63939,59337,61038,...,58601,62345,53891,53602,52109,52027,52392,55766,54796,57041
1,AL,243953,233910,240358,235720,246269,249452,260306,271789,264633,...,252887,253858,255557,244860,252383,246511,224442,236984,244935,237952
2,AR,142474,146067,146019,149592,147995,157852,155943,157748,159838,...,178727,174121,170622,169426,180744,175320,164343,172350,173393,169750
3,AZ,297677,296835,296714,296826,301398,310534,319900,329184,329560,...,323017,327685,330002,330238,327557,325399,310608,317891,337261,344621
4,CA,1195092,1309918,1282989,1244763,1355747,1295011,1308405,1377978,1419732,...,1253279,1286920,1226186,1187925,1247418,1192673,1124296,1156382,1193155,1157666


In [80]:
'Total energy consumption in the industrial sector, billion Btu'
industrial_use = industrial_use.drop(columns=[col for col in industrial_use.columns if str(col).isdigit() and int(col) < 2000])
industrial_use.head()

,State,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,AK,400347,408116,409411,400734,387948,416229,352071,354704,315931,...,327606,328407,327147,331033,331478,333595,362057,386830,429873,437648
1,AL,978320,833655,841950,855029,922250,912008,920116,908318,865795,...,846768,810102,803843,813582,829542,804116,757523,809632,770887,758508
2,AR,469550,440731,447699,437795,426905,423291,437513,433350,411202,...,408014,371113,375182,392153,406022,393740,368907,381366,379728,383533
3,AZ,217561,206781,197776,192187,218525,214966,220537,216154,227388,...,228055,226778,230320,221978,219663,218065,215250,214371,219705,215181
4,CA,1991971,2011063,1877339,1847872,1923879,1856848,1819106,1830417,1784350,...,1780682,1771261,1748974,1708736,1739402,1690656,1605591,1597289,1538137,1456739


In [81]:
'Total energy consumption in the transportation sector, billion Btu'
transportation_use = transportation_use.drop(columns=[col for col in transportation_use.columns if str(col).isdigit() and int(col) < 2000])
transportation_use.head()

,State,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,AK,219733,205457,209339,219243,265801,263620,265470,249625,214641,...,155101,162953,153610,155909,163853,164603,164911,193828,198746,203538
1,AL,472549,445445,472527,458057,493169,489525,496100,499549,467614,...,465265,480809,504534,498694,488553,500827,498127,555419,552282,543739
2,AR,293838,277439,285353,285258,288206,288729,288737,292672,285174,...,280120,276980,283493,285132,286161,288886,271194,284853,283659,287260
3,AZ,459904,475175,485228,500076,510211,532716,549271,541381,504995,...,517027,527180,540357,543632,554443,570394,508524,565481,549992,557787
4,CA,3016982,2966568,3129911,3040202,3192745,3278919,3327189,3354398,3144123,...,2796247,2862554,2967548,3045692,3059446,3058130,2353243,2800411,2917308,3035725


In [82]:
'Total energy consumption, billion Btu'
total_consumption = total_consumption.drop(columns=[col for col in total_consumption.columns if str(col).isdigit() and int(col) < 2000])
total_consumption.head()

,State,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,AK,735260,726418,723166,722838,765107,788012,738306,714615,642737,...,585727,603471,582349,590557,593777,595801,626207,684975,730277,746978
1,AL,2065056,1859587,1920029,1897040,2017765,2018912,2050932,2067485,1976589,...,1930039,1880980,1895651,1867860,1912733,1883685,1795668,1927858,1904247,1860171
2,AR,1124742,1078834,1095219,1083763,1069229,1094815,1103064,1108782,1082215,...,1108392,1043382,1036535,1043618,1102555,1077563,1012312,1057206,1058036,1049897
3,AZ,1289942,1302269,1298175,1319572,1372363,1411695,1457800,1466903,1433406,...,1424922,1443936,1465473,1465780,1477247,1495891,1439270,1482635,1512022,1526567
4,CA,7504612,7610078,7595049,7454129,7835668,7741970,7785762,7923319,7748640,...,7053028,7136949,7136403,7145251,7292387,7231426,6432232,6778725,6855767,6816746


In [83]:
'Real gross domestic product (GDP), million chained (2017) dollars'
real_gdp = real_gdp.drop(columns=[col for col in real_gdp.columns if str(col).isdigit() and int(col) < 2000])
real_gdp.head()

,State,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,AK,38428.1,40014.4,41904.7,41235.0,42880.1,44406.8,47546.2,50126.7,49926.6,...,53303.3,53681.1,53463.9,53550.9,52479.6,52377.5,50332.8,51454.1,50767.1,54059.7
1,AL,168695.3,168448.3,172430.4,178040.0,189913.7,197246.5,199923.9,202025.5,200967.4,...,206070.0,208950.3,212862.8,216615.5,220808.8,225272.8,222288.8,233726.6,238556.5,245354.7
2,AR,95509.8,95528.7,98556.6,102948.0,108581.8,112451.1,114890.4,114026.7,113628.0,...,121409.4,121532.5,123034.9,123882.6,126371.2,127220.0,128340.9,137463.6,139560.7,142860.6
3,AZ,224729.3,230885.6,238257.1,253743.1,264990.0,283806.2,296424.4,306611.6,302277.9,...,301721.2,308582.8,319008.4,333099.0,346398.3,359576.7,365027.7,395035.9,410228.4,422399.6
4,CA,1784320.6,1784567.7,1821509.5,1895287.6,1956745.5,2042141.0,2120435.4,2168940.4,2192331.8,...,2428675.7,2545979.5,2623711.7,2740550.3,2850970.3,2969609.0,2933320.2,3154188.6,3184007.8,3248656.6


In [84]:
'Total energy CO2 emissions, million metric tons CO2'
total_co2 = total_co2.drop(columns=[col for col in total_co2.columns if str(col).isdigit() and int(col) < 2000])
total_co2.head()

,State,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,AK,43.8,42.8,43.0,43.3,46.5,47.8,45.6,43.8,39.2,...,34.1,35.0,33.4,33.7,34.5,34.3,36.1,39.8,42.1,43.2
1,AL,142.4,133.3,138.4,139.9,142.1,143.6,145.8,147.3,139.4,...,122.6,118.9,113.5,108.2,112.0,106.0,98.1,108.4,109.5,101.0
2,AR,63.3,62.5,61.2,62.3,62.5,60.3,62.1,63.4,64.2,...,68.8,58.8,61.8,63.9,70.5,64.9,54.5,61.8,63.3,60.4
3,AZ,86.6,88.9,88.3,90.5,97.3,97.3,100.5,102.3,102.6,...,97.3,94.9,90.8,90.4,94.0,92.5,80.1,82.8,80.4,83.0
4,CA,382.5,385.8,384.2,374.9,392.7,389.5,398.0,402.6,384.2,...,343.2,347.3,348.9,352.3,355.0,354.6,299.6,321.9,324.1,324.3


In [85]:
'Carbon intensity of energy supply (CO2 emissions divided by total energy consumption less interstate flow of electricity), metric tons CO2 per billion Btu'											
carbon_intensity_energy = carbon_intensity_energy.drop(columns=[col for col in carbon_intensity_energy.columns if str(col).isdigit() and int(col) < 2000])
carbon_intensity_energy.head()

,State,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,AK,59.6,59.0,59.4,59.9,60.7,60.6,61.7,61.2,61.0,...,58.3,58.1,57.4,57.1,58.2,57.6,57.6,58.1,57.7,57.8
1,AL,59.7,59.7,59.5,60.2,59.0,59.7,59.5,59.3,57.4,...,51.4,50.1,49.4,47.7,48.4,46.4,44.8,46.1,46.8,44.6
2,AR,57.7,57.3,55.8,56.1,56.9,57.0,56.2,56.2,57.5,...,57.1,53.9,54.6,55.9,57.0,54.5,51.3,54.4,53.8,52.5
3,AZ,57.5,58.4,57.0,58.2,58.3,58.9,59.8,58.6,57.6,...,56.5,54.8,53.3,53.7,53.9,52.0,48.8,49.3,48.7,48.5
4,CA,55.9,56.3,56.4,56.0,56.5,56.1,56.6,56.5,56.1,...,54.5,54.4,54.0,53.8,54.5,53.8,52.0,52.3,51.8,50.3


In [86]:
'Renewable Portfolio Standard: required percentage of electricity that must come from renewable sources'

col_names = [
    "State", "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA",
    "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD",
    "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ",
    "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
    "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"
]


rps.columns = col_names
rps = rps.T
new_header = rps.iloc[0] 
rps.columns = new_header
rps.columns = [int(col) for col in rps.columns]
rps = rps.reset_index()
rps = rps.rename(columns={'index': 'State'})
rps = rps[1:] 
rps.head()

,State,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
1,AL,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00
2,AK,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00
3,AZ,0.0,0.0,0.0,0.0,0.0,0.71,1.42,2.13,2.84,...,8.52,9.23,9.94,10.65,11.36,12.07,12.78,13.49,14.2,14.91
4,AR,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00
5,CA,0.0,0.0,1.1,2.2,3.3,4.40,5.50,6.60,7.70,...,16.50,18.80,21.10,23.40,25.70,28.00,30.30,32.60,34.9,37.20


In [87]:
# Processing population datasets


# Mapping dictionary for the 50 states (with dot)
state_codes = {
    '.Alabama': 'AL', '.Alaska': 'AK', '.Arizona': 'AZ', '.Arkansas': 'AR',
    '.California': 'CA', '.Colorado': 'CO', '.Connecticut': 'CT', '.Delaware': 'DE',
    '.Florida': 'FL', '.Georgia': 'GA', '.Hawaii': 'HI', '.Idaho': 'ID',
    '.Illinois': 'IL', '.Indiana': 'IN', '.Iowa': 'IA', '.Kansas': 'KS',
    '.Kentucky': 'KY', '.Louisiana': 'LA', '.Maine': 'ME', '.Maryland': 'MD',
    '.Massachusetts': 'MA', '.Michigan': 'MI', '.Minnesota': 'MN', '.Mississippi': 'MS',
    '.Missouri': 'MO', '.Montana': 'MT', '.Nebraska': 'NE', '.Nevada': 'NV',
    '.New Hampshire': 'NH', '.New Jersey': 'NJ', '.New Mexico': 'NM', '.New York': 'NY',
    '.North Carolina': 'NC', '.North Dakota': 'ND', '.Ohio': 'OH', '.Oklahoma': 'OK',
    '.Oregon': 'OR', '.Pennsylvania': 'PA', '.Rhode Island': 'RI', '.South Carolina': 'SC',
    '.South Dakota': 'SD', '.Tennessee': 'TN', '.Texas': 'TX', '.Utah': 'UT',
    '.Vermont': 'VT', '.Virginia': 'VA', '.Washington': 'WA', '.West Virginia': 'WV',
    '.Wisconsin': 'WI', '.Wyoming': 'WY'
}

# Function to clean DataFrame
def clean_state_df(df):
    # Remove na
    df.dropna(inplace=True)
    # Rename first column to "State"
    df = df.rename(columns={df.columns[0]: 'State'})
    # Keep only rows starting with '.' and exclude D.C. and Puerto Rico
    df = df[df['State'].str.startswith('.') & 
            ~df['State'].isin(['.District of Columbia', '.Puerto Rico'])].copy()
    # Map full names to state codes
    df['State'] = df['State'].map(state_codes)
    # Reset index
    return df.reset_index(drop=True)

# Apply to both pop2 and pop3
pop1 = clean_state_df(pop1)
pop2 = clean_state_df(pop2)
pop3 = clean_state_df(pop3)

# Removing unwanted columns
pop1 = pop1.drop(columns=['?', '??', '???'])
pop2 = pop2.drop(columns=['Census', 'Estimates Base'])
pop3 = pop3.drop(columns=['Unnamed: 1'])

# Merginging into single pop df
pop_merged = pop1.merge(pop2, on='State', how='outer') \
                 .merge(pop3, on='State', how='outer')

# Then melt from wide to long
population = pop_merged.melt(id_vars='State', var_name='Year', value_name='population')

In [88]:
population.head()

,State,Year,population
0,AL,2000,4452173.0
1,AK,2000,627963.0
2,AZ,2000,5160586.0
3,AR,2000,2678588.0
4,CA,2000,33987977.0


In [89]:
import pandas as pd
from functools import reduce
# Inner merge on 'State'

# List all DataFrames
dfs = [
    (electricity_sales, 'electricity_sales'),
    (coal_use, 'coal_use'),
    (ng_use, 'natural_gas_use'),
    (petrol_use, 'petroleum_use'),
    (nuclear_use, 'nuclear_use'),
    (renewables_use, 'renewables_use'),
    (biomass_use, 'biomass_use'),
    (geothermal_use, 'geothermal_use'),
    (hydro_use, 'hydro_use'),
    (solar_use, 'solar_use'),
    (wind_use, 'wind_use'),
    (residential_use, 'residential_use'),
    (commercial_use, 'commercial_use'),
    (industrial_use, 'industrial_use'),
    (transportation_use, 'transportation_use'),
    (total_consumption, 'total_consumption'),
    (real_gdp, 'real_gdp'),
    (total_co2, 'total_co2'),
    (carbon_intensity_energy, 'carbon_intensity'),
    (rps, 'rps')
]

# Melt all DataFrames
melted_dfs = []
for df, feature_name in dfs:
    df_long = pd.melt(
        df,
        id_vars=['State'],
        var_name='Year',
        value_name=feature_name
    )
    melted_dfs.append(df_long)

# Merge all melted DataFrames on ['State', 'Year']
from functools import reduce
merged = reduce(lambda left, right: pd.merge(left, right, on=['State', 'Year'], how='inner'), melted_dfs)

# Merge population column
merged = pd.merge(merged, population, on=['State', 'Year'], how='outer')

In [90]:
merged.head()

,State,Year,electricity_sales,coal_use,natural_gas_use,petroleum_use,nuclear_use,renewables_use,biomass_use,geothermal_use,...,residential_use,commercial_use,industrial_use,transportation_use,total_consumption,real_gdp,total_co2,carbon_intensity,rps,population
0,AK,2000,5310,16455,437972,275255,0,5574,2069,86,...,48404,66776,400347,219733,735260,38428.1,43.8,59.6,0.0,627963.0
1,AL,2000,83524,904181,368472,559735,327143,223807,203782,62,...,370236,243953,978320,472549,2065056,168695.3,142.4,59.7,0.0,4452173.0
2,AR,2000,41611,267579,256114,357586,121516,92540,83466,192,...,218879,142474,469550,293838,1124742,95509.8,63.3,57.7,0.0,2678588.0
3,AZ,2000,61130,432810,208137,502693,316839,45505,13376,286,...,314800,297677,217561,459904,1289942,224729.3,86.6,57.5,0.0,5160586.0
4,CA,2000,244057,70012,2456438,3571465,366845,370066,164105,44038,...,1300566,1195092,1991971,3016982,7504612,1784320.6,382.5,55.9,0.0,33987977.0


In [93]:
# Energy sum of major fossil fuels and renewables (general)
merged['total_energy'] = merged['coal_use'] + merged['natural_gas_use'] + merged['nuclear_use'] + merged['petroleum_use'] + merged['biomass_use'] + merged['geothermal_use'] + merged['hydro_use'] +merged['solar_use'] +merged['wind_use']

# Computing percent contribution of each energy type
merged['%_coal'] = merged['coal_use'] / merged['total_energy']
merged['%_natural_gas'] = merged['natural_gas_use'] / merged['total_energy']
merged['%_petroleum'] = merged['petroleum_use'] / merged['total_energy']
merged['%_nuclear'] = merged['nuclear_use'] / merged['total_energy']

merged['%_biomass']    = merged['biomass_use']    / merged['total_energy']
merged['%_geothermal'] = merged['geothermal_use'] / merged['total_energy']
merged['%_hydro']      = merged['hydro_use']      / merged['total_energy']
merged['%_solar']      = merged['solar_use']      / merged['total_energy']
merged['%_wind']       = merged['wind_use']       / merged['total_energy']


In [94]:
# Validating that percentages sum to 1
merged['percent_sum'] = (
    merged['%_coal'] +
    merged['%_natural_gas'] +
    merged['%_petroleum'] +
    merged['%_nuclear'] +
    merged['%_biomass'] +
    merged['%_geothermal'] +
    merged['%_hydro'] +
    merged['%_solar'] +
    merged['%_wind']
)

# Check if any rows are significantly different from 1 (Summary statistics)
print(merged['percent_sum'].describe())
merged.drop('percent_sum', axis=1, inplace=True)

count    1.200000e+03
mean     1.000000e+00
std      9.682735e-17
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      1.000000e+00
Name: percent_sum, dtype: float64


In [95]:
# Creating lagged features (energy mix, GDP, energy intensity, policy columns)

merged.columns
# Sort for consistent lagging by state and year
merged = merged.sort_values(['State', 'Year'])

# Define columns to lag by category
lag_config = {
    # Energy mix variables (lag 1)
    # Changes in generation mix or fuel type affect emissions within the next reporting year
    'coal_use': 1,
    'natural_gas_use': 1,
    'petroleum_use': 1,
    'nuclear_use': 1,
    'renewables_use': 1,
    'biomass_use': 1,
    'geothermal_use': 1,
    'hydro_use': 1,
    'solar_use': 1,
    'wind_use': 1,

    # Economic and energy intensity indicators (lag 1)
    # Economic output influences emissions through next year’s industrial & consumption patterns
    'real_gdp': 1,
    'carbon_intensity': 1,
    'electricity_sales': 1,
    'total_consumption': 1,

    # Policy variables (lag 2)
    # Policy takes time to implement
    'rps': 2
}

# Create lagged features per state
for col, lag in lag_config.items():
    lagged_col = f'{col}_lag{lag}'
    # Ensure each state's data is lagged independently
    merged[lagged_col] = merged.groupby('State')[col].shift(lag)

In [96]:
merged.columns

Index(['State', 'Year', 'electricity_sales', 'coal_use', 'natural_gas_use',
       'petroleum_use', 'nuclear_use', 'renewables_use', 'biomass_use',
       'geothermal_use', 'hydro_use', 'solar_use', 'wind_use',
       'residential_use', 'commercial_use', 'industrial_use',
       'transportation_use', 'total_consumption', 'real_gdp', 'total_co2',
       'carbon_intensity', 'rps', 'population', 'total_energy', '%_biomass',
       '%_geothermal', '%_hydro', '%_solar', '%_wind', '%_coal',
       '%_natural_gas', '%_petroleum', '%_nuclear', 'percent_sum',
       'coal_use_lag1', 'natural_gas_use_lag1', 'petroleum_use_lag1',
       'nuclear_use_lag1', 'renewables_use_lag1', 'biomass_use_lag1',
       'geothermal_use_lag1', 'hydro_use_lag1', 'solar_use_lag1',
       'wind_use_lag1', 'real_gdp_lag1', 'carbon_intensity_lag1',
       'electricity_sales_lag1', 'total_consumption_lag1', 'rps_lag2'],
      dtype='object')

In [97]:
# Exporting to Excel
merged.to_excel("emissions_forecasting_data.xlsx", index=False)

In [108]:
# ------------------
# INVESTIGATING WHY RENEWABLE PERCENTAGES ARE LOW
# ------------------

# Check 2023 data
merged_2023 = merged[merged["Year"] == 2023]

# Are all your energy sources in the same units (billion Btu)?
print("Coal:", merged_2023["coal_use"].sum())
print("Natural Gas:", merged_2023["natural_gas_use"].sum())
print("Petroleum:", merged_2023["petroleum_use"].sum())
print("Nuclear:", merged_2023["nuclear_use"].sum())
print("Wind:", merged_2023["wind_use"].sum())
print("Solar:", merged_2023["solar_use"].sum())
print("Total Energy:", merged_2023["total_energy"].sum())

merged_2023 = merged[merged["Year"] == 2023]
calculated_renewables = (merged_2023["biomass_use"].sum() + 
                         merged_2023["geothermal_use"].sum() + 
                         merged_2023["hydro_use"].sum() + 
                         merged_2023["solar_use"].sum() + 
                         merged_2023["wind_use"].sum())
actual_renewables = merged_2023["renewables_use"].sum()

print(f"Calculated renewables: {calculated_renewables:,.0f}")
print(f"Actual renewables_use: {actual_renewables:,.0f}")
print(f"Difference: {actual_renewables - calculated_renewables:,.0f}")

# List of individual components
energy_cols = [
    "coal_use",
    "natural_gas_use",
    "petroleum_use",
    "nuclear_use",
    "biomass_use",
    "geothermal_use",
    "hydro_use",
    "solar_use",
    "wind_use"
]

# National totals
national_totals = merged_2023[energy_cols].sum()
national_total_energy = national_totals.sum()

# Percent of total from each energy source
percent_2023 = (national_totals / national_total_energy) * 100

print("\nPercent Use of Each Energy Source in 2023 (Nationwide):")
print(percent_2023.round(2))

Coal: 8169672
Natural Gas: 33582870
Petroleum: 35443065
Nuclear: 8098972
Wind: 1436936
Solar: 879535
Total Energy: 93440918
Calculated renewables: 8,146,339
Actual renewables_use: 8,146,332
Difference: -7

Percent Use of Each Energy Source in 2023 (Nationwide):
coal_use            8.74
natural_gas_use    35.94
petroleum_use      37.93
nuclear_use         8.67
biomass_use         5.22
geothermal_use      0.13
hydro_use           0.89
solar_use           0.94
wind_use            1.54
dtype: float64


The calculations for the renewable percentage columns appear correct. The sum of each individual renewable type is almost identical to the cumulative renewable column (which was directly collected from a dataset). The percent breakdown of wind is likely low because we are computing the composition of total primary energy, not just for electricity, which causes petroleum and natural gas to dominate.